In [2]:
# ============================================================
# REGRESIÓN LINEAL MÚLTIPLE — VERSIÓN SENCILLA PARA PRINCIPIANTES
# Predicción de "Conversiones" en campañas de marketing
# ============================================================

# 0) Objetivo de este cuaderno (en una línea):
# Entrenar un modelo lineal sencillo que use varias columnas (X) para predecir Conversiones (y).

# --------------------------
# 1) Importar librerías
# --------------------------
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt

# Configuración mínima
np.random.seed(42)


In [3]:
# --------------------------
# 2) Cargar datos
# --------------------------
# Reemplaza la ruta por tu archivo CSV
df = pd.read_csv("/content/marketing_campaigns_1000_rows.csv")


In [4]:
df

,Conversiones,Inversion_USD,Puntaje_Creativo,Precision_Targeting,CTA_Claro,Num_Hashtags,Hora_Publicacion,Longitud_Texto,Dia_Semana
0,155,500,8,90,1,5,18.0,280,3
1,68,250,5,60,0,15,9.0,450,5
2,210,800,9,95,1,2,20.5,150,2
3,45,300,4,50,0,8,11.0,320,7
4,130,600,7,80,1,12,14.0,220,1


In [ ]:
# --------------------------
# 2) Cargar datos
# --------------------------
# Reemplaza la ruta por tu archivo CSV
df = pd.read_csv("/content/marketing_campaigns_1000_rows.csv")

print("Dimensiones:", df.shape)
print("Columnas:", list(df.columns))
print("\nVista rápida:")
print(df.head())

In [5]:
# --------------------------
# 3) Dividir datos temprano
# --------------------------
# y = variable objetivo; X = resto de columnas
y = df["Conversiones"]
X = df.drop(columns=["Conversiones"])

# 80% entrenamiento, 20% prueba (mezcla aleatoria controlada por seed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

print("\nTamaño entrenamiento:", X_train.shape)
print("Tamaño prueba:", X_test.shape)



Tamaño entrenamiento: (4, 8)
Tamaño prueba: (1, 8)


In [6]:
# ----------------------------------------------------------
# 4) Preparar datos (imputación simple + codificación sencilla)
# ----------------------------------------------------------
# Idea: dejar todo en formato numérico y sin huecos (NaN).
# - Numéricas: rellenar missing con la MEDIANA (robusta y fácil)
# - Categóricas: convertir a dummies (one-hot) con pandas.get_dummies

# Detectar columnas por tipo (búsqueda simple y directa)
numeric_cols = X_train.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = X_train.select_dtypes(include=["object", "category"]).columns.tolist()

# Copias para no alterar los originales
X_train_prep = X_train.copy()
X_test_prep  = X_test.copy()

# a) Imputación numérica (mediana)
for col in numeric_cols:
    mediana = X_train_prep[col].median()
    X_train_prep[col] = X_train_prep[col].fillna(mediana)
    X_test_prep[col]  = X_test_prep[col].fillna(mediana)

# b) One-Hot Encoding para categóricas (train -> define columnas)
X_train_prep = pd.get_dummies(X_train_prep, columns=categorical_cols, drop_first=True)

# c) Aplicar las mismas columnas al set de prueba (alineación estricta)
X_test_prep  = pd.get_dummies(X_test_prep,  columns=categorical_cols, drop_first=True)
X_test_prep  = X_test_prep.reindex(columns=X_train_prep.columns, fill_value=0)

print("\nColumnas finales tras preparación:", X_train_prep.shape[1])


Columnas finales tras preparación: 8


In [7]:
# ----------------------------------------------------------
# 5) Entrenar un modelo lineal básico
# ----------------------------------------------------------
modelo = LinearRegression()
modelo.fit(X_train, y_train)

LinearRegression()

In [15]:
# ----------------------------------------------------------
# 6) Evaluación simple en entrenamiento y prueba
# ----------------------------------------------------------
# Predicciones
y_train_pred = modelo.predict(X_train)
y_test_pred  = modelo.predict(X_test)

In [16]:
# Métricas
rmse_train = mean_squared_error(y_train, y_train_pred)
mae_train  = mean_absolute_error(y_train, y_train_pred)
r2_train   = r2_score(y_train, y_train_pred)

In [19]:
rmse_train

0.0

In [12]:
r2_test

nan

In [ ]:
# MAPE (porcentaje de error absoluto medio). Nota: evita dividir por cero.
# Para mantenerlo muy simple, se calcula sobre valores > 0.
y_test_safe = y_test.replace(0, np.nan)
mape_test = np.mean(np.abs((y_test_safe - y_pred_test) / y_test_safe)) * 100

print("\n=== MÉTRICAS ENTRENAMIENTO ===")
print("RMSE:", round(rmse_train, 3))
print("MAE :", round(mae_train, 3))
print("R2  :", round(r2_train, 3))

print("\n=== MÉTRICAS PRUEBA (importan más) ===")
print("RMSE:", round(rmse_test, 3))
print("MAE :", round(mae_test, 3))
print("R2  :", round(r2_test, 3))
print("MAPE (aprox):", round(mape_test, 2), "%")


In [ ]:
 7) Visualizaciones básicas (diagnóstico rápido)
# ----------------------------------------------------------
# a) Reales vs. Predichos (ideal: puntos cerca de la diagonal)
plt.figure(figsize=(6,5))
plt.scatter(y_test, y_pred_test, alpha=0.6)
min_val = min(y_test.min(), y_pred_test.min())
max_val = max(y_test.max(), y_pred_test.max())
plt.plot([min_val, max_val], [min_val, max_val], "--")
plt.xlabel("Conversiones reales")
plt.ylabel("Conversiones predichas")
plt.title("Reales vs. Predichos (Prueba)")
plt.grid(alpha=0.3)
plt.show()

# b) Residuales (errores) — deberían verse alrededor de 0
residuals = y_test - y_pred_test
plt.figure(figsize=(6,5))
plt.scatter(y_pred_test, residuals, alpha=0.6)
plt.axhline(0, linestyle="--")
plt.xlabel("Predicción")
plt.ylabel("Error (Residual)")
plt.title("Residual vs. Predicción (Prueba)")
plt.grid(alpha=0.3)
plt.show()


In [ ]:
# ----------------------------------------------------------
# 8) "Importancia" de variables en un modelo lineal
# ----------------------------------------------------------
# En modelos lineales, el peso de cada columna es su coeficiente.
# Para comparar escalas, miramos el valor absoluto (tamaño del efecto).
coef = pd.Series(modelo.coef_, index=X_train_prep.columns, name="coeficiente")
importancia = coef.abs().sort_values(ascending=False)

print("\nTop 10 variables (por valor absoluto del coeficiente):")
print(importancia.head(10))


In [ ]:
# ----------------------------------------------------------
# 9) Guardar el modelo (opcional, pero útil para reutilizar)
# ----------------------------------------------------------
# Se guarda solo si lo necesitas más adelante.
# Descomenta estas dos líneas para activarlo:
# import joblib
# joblib.dump(modelo, "modelo_lineal_marketing.pkl")

# Probado el modelo

In [ ]:
X_nuevo = pd.DataFrame([{
    # Ejemplos (cámbialos por tus variables reales):
    "impresiones": 120000,
    "clics": 3800,
    "costo": 950.0,
    "canal": "Search",
    "pais": "PA",
    "dispositivo": "Mobile"
}])

In [ ]:
# 1) Alinear columnas crudas igual que en el entrenamiento
feature_names = list(X_train.columns)   # mismas columnas y orden
X_nuevo = X_nuevo.reindex(columns=feature_names)

# 2) Transformar con el MISMO preprocesador ya AJUSTADO
X_nuevo_prep = preprocess.transform(X_nuevo)

# 3) Predecir con el mismo modelo (sin re-entrenar)
y_pred_nuevo = modelo.predict(X_nuevo_prep)

# 4) Ver el resultado
import pandas as pd
pd.DataFrame({"prediccion": y_pred_nuevo})
